<a href="https://www.nvidia.com/dli"> <img src="../images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 3.0 Build a Named Entity Recognizer
### (NVIDIA NeMo v1.0)

In this notebook, you'll build an NER (named entity recognition) application that finds disease names in medical disease abstracts. The model does not "search" for names from a list, but rather "recognizes" that certain words are disease references from the context of the language. 

**[3.1 Token Classification from the Command Line](#3.1-Token-Classification-from-the-Command-Line)**<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1.1 Data Input](#3.1.1-Data-Input)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.1.1.1 IOB Tagging](#3.1.1.1-IOB-Tagging)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1.2 Configuration File](#3.1.2-Configuration-File)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1.3 Hydra-Enabled Python Scripts](#3.1.3-Hydra-Enabled-Python-Scripts)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1.4 Exercise: Train the Model](#3.1.4-Exercise:-Train-the-Model)<br>
**[3.2 Domain-Specific Training](#3.2-Domain-Specific-Training)**<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2.1 Visualize the Results with TensorBoard](#3.2.1-Visualize-the-Results-with-TensorBoard)<br>
**[3.3 Evaluation](#3.3-Evaluation)**<br>
**[3.4 Inference](#3.4-Inference)**<br>

For the NER task, you'll follow the same basic steps as in the text classification task to build your project, train it, and test it.  This time, however, you'll train a classifier on the *domain-specific* BioMegatron language model.  BioMegatron is a [BERT](https://arxiv.org/abs/1810.04805)-like [Megatron-LM](https://arxiv.org/pdf/1909.08053.pdf) model pre-trained on a large biomedical text corpus ([PubMed](https://pubmed.ncbi.nlm.nih.gov/) abstracts and full-text commercial use collection).  We can expect to have better performance compared to the general language models, because our disease dataset is from the same biomedical domain.

There are some alternatives of BioMegatron, most notably [BioBERT](https://arxiv.org/abs/1901.08746). Compared to BioBERT, BioMegatron is larger by model size and pre-trained on larger text corpus.

---
# 3.1 Token Classification from the Command Line
The question we want to answer is:

**Given sentences from medical abstracts, what diseases are mentioned?**<br>

Recall the NLP models available with NeMo:

In [ ]:
!tree nemo/examples/nlp -L 1

We use the [token classification](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/token_classification.html) model for NER because we are classifying at the "token" level, in this case classifying words related to diseases.

## 3.1.1 Data Input
As we saw in the [1.0 Explore the Data](010_ExploreData.ipynb) notebook, the dataset for the NER project is made up of sentences with IOB tagging for disease names, where each word in a sentence is tagged as inside, outside, or the beginning of a named entity. 

The training text and label files are `text_train.txt` and `labels_train.txt`, respectively.  The validation and test files follow a similar naming pattern. Verify the location of the data files. 

In [ ]:
NER3_DATA_DIR = '/dli/task/data/NCBI_ner-3'
!ls -lh $NER3_DATA_DIR

In [ ]:
# Take a look at the data
print("*****\ntext_test.txt sample\n*****")
!head -n 3 $NER3_DATA_DIR/text_test.txt
print("\n*****\nlabels_test.txt sample\n*****")
!head -n 3 $NER3_DATA_DIR/labels_test.txt

### 3.1.1.1 IOB Tagging
Recall that the sentences and labels in the NER dataset map to each other with _inside, outside, beginning (IOB)_ tagging.
This mechanism can be used in a general way for multiple named entity types:
* B-{CHUNK_TYPE} – for the word in the Beginning chunk
* I-{CHUNK_TYPE} – for words Inside the chunk
* O – Outside any chunk

In our case, we are only looking for "disease" as our entity (or chunk) type, so we don't need to identify beyond the three classes: I, O, and B.
**Three classes**
* B - Beginning of disease name
* I - Inside word of disease name
* O - Outside of all disease names

```text
Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor .
O              O  O    O O O         O  O   B           I         I    I      O          O  
```

These are defined in our `labels.csv` file:

In [ ]:
!head $NER3_DATA_DIR/label_ids.csv

If we were looking for two kinds of named entities, such as nouns and verbs in a parts-of-speech analysis, we would use a five-class IOB scheme:<br>
**Five classes**
* B-N - Beginning of noun word or phrase
* I-N - Inside noun word or phrase
* B-V - Beginning of verb word or phrase
* I-V - Inside verb word or phrase
* O   - Outside all nouns and verbs

If you are intereested in learning more, take a look at [this paper](http://cs229.stanford.edu/proj2005/KrishnanGanapathy-NamedEntityRecognition.pdf) on the subject.

The NCBI_ner-3 disease data is in the correct format for token classification as described in the [documentation](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/token_classification.html#data-input-for-token-classification-model), so we are ready to look at the configuration file.

## 3.1.2 Configuration File
Look at more detail for the NeMo token classification directory:

In [ ]:
TC_DIR = "/dli/task/nemo/examples/nlp/token_classification"
!tree $TC_DIR

The config file for NER, `token_classification_config.yaml`, specifies model, training, and experiment management details, such as file locations, pretrained models, and hyperparameters.  This is the same general pattern used in the text classification configuration file.  We'll take a look at the details of each section using the `OmegaConf` tool introduced in the text classification project.

In [ ]:
from omegaconf import OmegaConf

CONFIG_DIR = "/dli/task/nemo/examples/nlp/token_classification/conf"
CONFIG_FILE = "token_classification_config.yaml"

config = OmegaConf.load(CONFIG_DIR + "/" + CONFIG_FILE)

# print the model section
print(OmegaConf.to_yaml(config.model))

In [ ]:
# complete list of supported BERT-like models
from nemo.collections import nlp as nemo_nlp
nemo_nlp.modules.get_pretrained_lm_models_list()

In the `model` section, a path for `dataset.data_dir` that contains all the data files is required.  The actual file names we are using already conform to the default values, so we don't need to override those.

For our first try, we can override `language_model.pretrained_model_name` to `bert-base-cased`, so we can compare the results to the domain-specific `biomegatron-bert-345m-cased` in another experiment.  Since we will need to conserve memory space to run BioMegatron, we will go ahead and reduce `dataset.max_seq_length` and the `batch_size`.

In [ ]:
# print the trainer section
print(OmegaConf.to_yaml(config.trainer))

For efficiency, we can set the `amp_level` to 'O1'.  Since the language models we are going to compare are large and take a long time to run, we will override the `max_epochs` to a small number. 

In [ ]:
# print the experiment manager section
print(OmegaConf.to_yaml(config.exp_manager))

There is no need to change the `exp_manger` default settings for now.

###### 3.1.3 Hydra-Enabled Python Scripts
The Python scripts, `token_classification_train.py` and `token_evaluate.py`, encapsulate everything needed to run a token classification experiment defined by the configuration file.  Training and evaluation are expected to be run separately in this case.  As with text classification, both scripts employ Facebook's [Hydra](https://hydra.cc/) tool for configuration management, which allows the entire experiment to be run from the command line, overriding config file values as needed.

To recap, the parameters we need to change or override are:

* `model.language_model.pretrained_model_name`: set to 'bert-base-cased'
* `model.dataset.data_dir`: set to /dli/task/data/NCBI_ner-3
* `model.dataset.max_seq_length`: 64
* `model.train_ds.batch_size`: set to 32
* `model.val_ds.batch_size`: set to 32
* `model.test_ds.batch_size`: set to 32
* `trainer.amp_level`: set to "O1"
* `trainer.max_epochs`: set to 3

## 3.1.4 Exercise: Train the Model
Run the training script, `token_classification_train.py` just as you ran similar experiments in text classification notebook.   

The new values for overrides are provided for you in the cell below.  Add the command with appropriate overrides and run the cell.  If you get stuck, refer to the [solution](solutions/ex3.1.4.ipynb).

In [ ]:
%%time
# The training takes about 2 minutes to run
   
TOKEN_DIR = "/dli/task/nemo/examples/nlp/token_classification"

# set the values we want to override
PRETRAINED_MODEL_NAME = 'bert-base-cased'
DATA_DIR = '/dli/task/data/NCBI_ner-3'
MAX_SEQ_LENGTH = 64
BATCH_SIZE = 32
AMP_LEVEL = 'O1'
MAX_EPOCHS = 3

# Override the config values in the command line
# FIXME

How were the results?  Your log should have included something like:

```
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         99.34      99.35      99.34      21648
    B (label_id: 1)                                         85.86      89.21      87.50        769
    I (label_id: 2)                                         91.74      89.00      90.35       1073
    -------------------
    micro avg                                               98.54      98.54      98.54      23490
    macro avg                                               92.31      92.52      92.40      23490
    weighted avg                                            98.55      98.54      98.55      23490
    
Epoch 2: 100%|█| 199/199 [00:15<00:00, 12.45it/s, loss=0.0251, v_num=4-43, val_l
```

---
# 3.2 Domain-Specific Training

Try another experiment, this time overriding the `model.language_model.pretrained_model_name` with `biomegatron-bert-345m-cased`.  This is a large model with 345 million parameter.  Therefore, it takes longer to run.

In [ ]:
%%time
# The training takes about 5-6 minutes to run
   
TOKEN_DIR = "/dli/task/nemo/examples/nlp/token_classification"

# set the values we want to override
PRETRAINED_MODEL_NAME = 'biomegatron-bert-345m-cased'
DATA_DIR = '/dli/task/data/NCBI_ner-3'
MAX_SEQ_LENGTH = 64
BATCH_SIZE = 32
AMP_LEVEL = 'O1'
MAX_EPOCHS = 3

# Override the config values in the command line
!python $TOKEN_DIR/token_classification_train.py \
        model.language_model.pretrained_model_name=$PRETRAINED_MODEL_NAME \
        model.dataset.data_dir=$DATA_DIR \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.batch_size=$BATCH_SIZE \
        model.validation_ds.batch_size=$BATCH_SIZE \
        model.test_ds.batch_size=$BATCH_SIZE \
        trainer.amp_level=$AMP_LEVEL \
        trainer.max_epochs=$MAX_EPOCHS

## 3.2.1 Visualize the Results with TensorBoard
The [experiment manager](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/core.html?highlight=tensorboard#experiment-manager) saves results for viewing with TensorBoard. Execute the following cell to create a link to TensorBoard for your instance, then click on the link to open Tensorboard in a tab on your browser.

In [ ]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

To compare the performance of the models you've run, select the "f1" scaler.  You can see all the models compared together or select individual models for comparison.  In this example comparison, five epochs were run.  The orange line shows results from the `bert-base-cased` model and the blue line is the `biomegatron-bert-345m-cased` model.  The BioMegatron model does quite well very quickly, as it is better able to discern the disease names. It still has a slightly higher f1 after five epochs. The model you choose for your own project depends on your constraints in memory, time, and performance requirements.  Note that your results may vary from the example due to randomness in the learning algorithm.

<img src="images/tensorboard_02.png" width=800px>

---
# 3.3 Evaluation

In [ ]:
# Restart the kernel
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

To evaluate the model over the test set, we must specify the location of the `.nemo` trained model. Each experiment runs results in a time-stamped directory under `nemo_experiments`.  If we drill down, we can find the `checkpoints` folder where the final `token_classification_model.nemo` resides. In the next cell, a bit of Python logic is used to capture a list of models, and identify the latest one.

In [ ]:
import glob

nemo_model_paths = glob.glob('nemo_experiments/token_classification_model/*/checkpoints/*.nemo')

# Sort newest first
nemo_model_paths.sort(reverse=True)
print("The latest model is \n{}".format(nemo_model_paths[0]))

There are a couple of ways to run an evaluation over the test set:
1. Execute `token_classification_evaluate.py` with the same overrides, plus an override for the `pretrained_model`, which must be in `.nemo` format.

```text
   !python $TOKEN_DIR/token_classification_evaluate.py \
        model.dataset.data_dir=$DATA_DIR \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.batch_size=$BATCH_SIZE \
        model.validation_ds.batch_size=$BATCH_SIZE \
        model.test_ds.batch_size=$BATCH_SIZE \
        model.language_model.pretrained_model_name=$PRETRAINED_MODEL_NAME \
        pretrained_model=$LATEST_MODEL
```
        
2. Instantiate the model by restoring the trained model checkpoint and execute a NeMo method to evaluate the test set.<br>
   This is the method we will step through.

In [ ]:
# Instantiate the model by restoring from the .nemo checkpoint
from nemo.collections import nlp as nemo_nlp

LATEST_MODEL = nemo_model_paths[0]
model = nemo_nlp.models.TokenClassificationModel.restore_from(LATEST_MODEL)

Evaluate the model with the test set using the `evaluate_from_file` method.  Set the `add_confusion_matrix` to True to get a nice visual representation of how well the model did.

In [ ]:
import os.path

DATA_DIR = '/dli/task/data/NCBI_ner-3'
OUTPUT_DIR = '/dli/task/nemo_experiments/token_classification_model/logs'
model.evaluate_from_file(
    text_file=os.path.join(DATA_DIR, 'text_test.txt'),
    labels_file=os.path.join(DATA_DIR, 'labels_test.txt'),
    output_dir=OUTPUT_DIR,
    add_confusion_matrix=True,
    normalize_confusion_matrix=True,
    batch_size=1
)

The results should look something like:

```
[NeMo I 2021-06-29 00:42:16 token_classification_model:499]                  precision    recall  f1-score   support
    
    O (label id: 0)     0.9958    0.9910    0.9934     22450
    B (label id: 1)     0.8886    0.9135    0.9009       960
    I (label id: 2)     0.8724    0.9374    0.9038      1087
    
           accuracy                         0.9856     24497
          macro avg     0.9189    0.9473    0.9327     24497
       weighted avg     0.9861    0.9856    0.9858     24497


The final confusion matrix visualization shows a bright diagonal, indicating that the predicted label matched the true label with high accuracy for all the label types (IOB).
```

<img src="images/ner_confusion_matrix.png">

---
# 3.4 Inference
To run inference on a list of queries, use the same model already loaded with the `add_predictions` method.

In [ ]:
queries = ["Clustering of missense mutations in the ataxia - telangiectasia gene in a sporadic T - cell leukaemia . ",
    "Ataxia - telangiectasia ( A - T ) is a recessive multi - system disorder caused by mutations in the ATM gene at 11q22 - q23 ( ref . 3 ) . ",
    "The risk of cancer , especially lymphoid neoplasias , is substantially elevated in A - T patients and has long been associated with chromosomal instability . ",
    "By analysing tumour DNA from patients with sporadic T - cell prolymphocytic leukaemia ( T - PLL ) , a rare clonal malignancy with similarities to a mature T - cell leukaemia seen in A - T , we demonstrate a high frequency of ATM mutations in T - PLL . ",
    "In marked contrast to the ATM mutation pattern in A - T , the most frequent nucleotide changes in this leukaemia were missense mutations . ",
    "These clustered in the region corresponding to the kinase domain , which is highly conserved in ATM - related proteins in mouse , yeast and Drosophila . ",
    "The resulting amino - acid substitutions are predicted to interfere with ATP binding or substrate recognition . ",
    "Two of seventeen mutated T - PLL samples had a previously reported A - T allele . ",
    "In contrast , no mutations were detected in the p53 gene , suggesting that this tumour suppressor is not frequently altered in this leukaemia . ",
    "Occasional missense mutations in ATM were also found in tumour DNA from patients with B - cell non - Hodgkins lymphomas ( B - NHL ) and a B - NHL cell line . "
          ]

In [ ]:
results = model.add_predictions(queries, output_file='predictions.txt')

In [ ]:
!cat predictions.txt

---
<h2 style="color:green;">Congratulations!</h2>

You've mastered NeMo and learned:
* How to build a named entity recognizer
* How to apply a domain-specific model
* How to test an NER model with queries from a checkpoint

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>